In [ ]:
!pip install kaleido==0.2.1

In [ ]:
# Import the sqlite3 library to work with an in-memory SQLite database
import sqlite3
conn = sqlite3.connect(':memory:')

# Import pandas for data manipulation and analysis
import pandas as pd

# Import Plotly Express and Graph Objects for creating interactive visualizations
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

from google.colab import files

In [ ]:
# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Define the file path to the dataset stored in Google Drive
file_path = '/content/drive/MyDrive/devra-ai-google-keyword-search-performance.csv'

In [ ]:
# Load the CSV file into a pandas DataFrame for analysis
df = pd.read_csv(file_path)

In [ ]:
# Rename columns for clarity and consistency:
df.rename(columns={'Impr.': 'Impressions'}, inplace=True)
df.rename(columns={'Avg. CPC': 'Avg CostPerClick'}, inplace=True)

In [ ]:
# Load the cleaned DataFrame into an in-memory
# SQLite database as a table named 'data'
df.to_sql('data', conn, index=False)

14402

In [ ]:
# calculating metrics that matter to marketers and adding as columns

df['ClickThruRate'] = df['Clicks'] / df['Impressions']
df['TotalCost'] = df['Clicks'] * df['Avg CostPerClick']
df.head()

,Search term,Match type,Impressions,Clicks,Currency code,Avg CostPerClick,Keyword,ClickThruRate,TotalCost
0,copilot,Broad match,2060,47,USD,0.10,coding ai,0.022816,4.70
1,inteligencia artificial,Broad match,694,20,USD,0.24,ai programs,0.028818,4.80
2,blackbox ai,Phrase match (close variant),670,25,USD,0.20,ai software,0.037313,5.00
3,copilot,Broad match,388,2,USD,0.31,AI software tools,0.005155,0.62
4,inteligência artificial,Broad match,312,23,USD,0.31,ai programs,0.073718,7.13


In [ ]:
df.to_sql('data', conn, index=False, if_exists='replace')

14402

In [ ]:
keyword_clicks_query = """
SELECT keyword, SUM(clicks) AS TotalClicks
FROM data
GROUP BY keyword
ORDER BY TotalClicks DESC;
"""
keyword_clicks = pd.read_sql_query(keyword_clicks_query, conn)
keyword_clicks.head()

# 'ai software' was the highest-performing keyword with 1100 clicks, followed by 'ai programs' and 'AI software tools'.
# These insights help prioritize which keywords drive the most engagement and should be scaled or analyzed further.

,Keyword,TotalClicks
0,ai software,1100
1,ai programs,279
2,AI software tools,226
3,ai apps,222
4,coding ai,199


In [ ]:
# Grouping performance metrics by match type to compare total clicks, impressions, cost, and engagement.
# Sorting match types first by AvgCTR (to prioritize ad engagement), and then
# by TotalCost (to see which match types drive the most expensive traffic).
# This helps compare performance vs. cost efficiency across match types.

match_type_query = """
SELECT
  "Match type",
  SUM(Clicks) AS TotalClicks,
  SUM(Impressions) AS TotalImpressions,
  AVG(ClickThruRate) AS AvgCTR,
  AVG("Avg CostPerClick") AS AvgCPC,
  SUM(TotalCost) AS TotalCost
FROM data
GROUP BY "Match type"
ORDER BY AVG(ClickThruRate) DESC, SUM(TotalCost) DESC;
"""
match_type = pd.read_sql_query(match_type_query, conn)
match_type.head()

# Match type performance analysis:
# - 'Exact match (close variant)' had the highest AvgCTR (0.0923) and low AvgCPC — strong performer.
# - 'Exact match' had low CTR (0.0331) and the highest AvgCPC — inefficient and expensive.
# - 'Broad match' had the highest impressions but lower CTR — wide reach, but less engagement.
# - 'Phrase match (close variant)' also had high impressions with modest CTR — could be refined.

,Match type,TotalClicks,TotalImpressions,AvgCTR,AvgCPC,TotalCost
0,Exact match (close variant),17,206,0.092344,0.031075,4.01
1,Broad match,2037,30833,0.067925,0.033042,561.76
2,Phrase match (close variant),429,7686,0.059790,0.031619,130.50
3,Exact match,29,403,0.033079,0.070370,9.26
4,Phrase match,9,257,0.026729,0.011088,1.63


In [ ]:
df['Flag_Low_CTR'] = df['ClickThruRate'] < 0.02
df['Flag_High_CPC'] = df['Avg CostPerClick'] > 1.50
df['Flag_Wasted_Reach'] = (df['Impressions'] > 1000) & (df['ClickThruRate'] < 0.02)
df.head()

,Search term,Match type,Impressions,Clicks,Currency code,Avg CostPerClick,Keyword,ClickThruRate,TotalCost,Flag_Low_CTR,Flag_High_CPC,Flag_Wasted_Reach
0,copilot,Broad match,2060,47,USD,0.10,coding ai,0.022816,4.70,False,False,False
1,inteligencia artificial,Broad match,694,20,USD,0.24,ai programs,0.028818,4.80,False,False,False
2,blackbox ai,Phrase match (close variant),670,25,USD,0.20,ai software,0.037313,5.00,False,False,False
3,copilot,Broad match,388,2,USD,0.31,AI software tools,0.005155,0.62,True,False,False
4,inteligência artificial,Broad match,312,23,USD,0.31,ai programs,0.073718,7.13,False,False,False


In [ ]:
df.to_sql('data', conn, index=False, if_exists='replace')

14402

In [ ]:
Low_CTR_query = """
SELECT Keyword, COUNT(*) as LowCTR_Flagged_Count
FROM data
WHERE Flag_Low_CTR = 1
GROUP BY Keyword;
"""
Low_CTR = pd.read_sql_query(Low_CTR_query, conn)
Low_CTR.head()

,Keyword,LowCTR_Flagged_Count
0,AI code analysis,22
1,AI code completion,2
2,AI code documentation,10
3,AI code enhancement,55
4,AI code learning,36


In [ ]:
High_CPC_query = """
SELECT Keyword, COUNT(*) as HighCPC_Flagged_Count
FROM data
WHERE Flag_High_CPC = 1
GROUP BY Keyword;
"""
High_CPC = pd.read_sql_query(High_CPC_query, conn)
High_CPC.head()

,Keyword,HighCPC_Flagged_Count
0,ai software,5
1,ai tools,1


In [ ]:
Wasted_reach_query = """
SELECT Keyword, COUNT(*) as Wasted_reach_count
FROM data
WHERE Impressions > 500 AND ClickThruRate < 0.04
GROUP BY Keyword;
"""
Wasted_reach = pd.read_sql_query(Wasted_reach_query, conn)
Wasted_reach.head()

,Keyword,Wasted_reach_count
0,ai programs,1
1,ai software,1
2,coding ai,1


In [ ]:
df_CTR_CPC_plot = df[['Keyword', 'ClickThruRate', 'Avg CostPerClick', 'Impressions', 'Match type']]
df_CTR_CPC_plot.head()

,Keyword,ClickThruRate,Avg CostPerClick,Impressions,Match type
0,coding ai,0.022816,0.10,2060,Broad match
1,ai programs,0.028818,0.24,694,Broad match
2,ai software,0.037313,0.20,670,Phrase match (close variant)
3,AI software tools,0.005155,0.31,388,Broad match
4,ai programs,0.073718,0.31,312,Broad match


In [ ]:
match_types = df_CTR_CPC_plot['Match type'].unique()

traces = []
for match in match_types:
  filtered_df = df_CTR_CPC_plot[df_CTR_CPC_plot['Match type'] == match].copy()
  filtered_df['Impressions_for_size'] = filtered_df['Impressions'].copy()
  filtered_df['AvgCostPerClick'] = '$' + filtered_df['Avg CostPerClick'].round(2).astype(str)
  filtered_df['ClickThruRate'] = filtered_df['ClickThruRate'].round(2)

  if (filtered_df['Impressions'] < 10).any():
    filtered_df.loc[filtered_df['Impressions_for_size'] < 50, 'Impressions_for_size'] += 50

  fig_temp = px.scatter(
    data_frame = filtered_df,
    x = 'Avg CostPerClick',
    y = 'ClickThruRate',
    hover_name = 'Keyword',
    size = 'Impressions_for_size',
    hover_data={'Avg CostPerClick': False,
                'AvgCostPerClick': True,
                'Impressions': True,
                'Impressions_for_size': False,
                },
    color_discrete_sequence = ['navy'],
  )
  trace = fig_temp.data[0]
  trace.name = match
  traces.append(trace)

fig = go.Figure(data = traces)

match_descriptions = {
    'Exact match': 'The search term matches the keyword exactly.',
    'Phrase match': 'The search term contains the keyword or a close variation in sequence.',
    'Broad match': 'The search term is loosely related to the keyword, providing a wider reach.',
    'Phrase match (close variant)': 'The search term contains a close variation of the keyword in sequence.',
    'Exact match (close variant)': 'The search term closely matches the keyword with the same meaning or intent.'
}

buttons = []
for i, match in enumerate(match_types):
    visible = [j == i for j in range(len(match_types))]  # Only one trace visible at a time
    buttons.append(dict(
        label=match,
        method='update',
        args=[
    {'visible': visible},
    {
        'title': f"Match Type: {match}",
        'annotations': [dict(
            x=0.5,
            y=1.08,
            xref='paper',
            yref='paper',
            text=match_descriptions[match],
            showarrow=False,
            xanchor='center',
            align='center',
            font=dict(size=14)
        )]
    }
]
    ))

fig.update_layout(
    title=dict(
        text = 'Click Thru Rate vs Avg Cost Per Click by Match Type',
        x=0.48,
        xanchor='center',
        font=dict(size=24)
    ),
    margin=dict(t=100),
    updatemenus=[dict(
        buttons=buttons,
        direction='down',
        showactive=True,
        x=1.1,
        y=1.2
    )],
    xaxis_title='Avg CostPerClick',
    yaxis_title='ClickThruRate'
)

# Display, save, and download plot
fig.show()
fig.write_html("CTR_vs_AvgCPC.html")
files.download("CTR_vs_AvgCPC.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_bar_chart = df.groupby('Match type')['ClickThruRate'].mean().reset_index()
df_bar_chart = df_bar_chart.sort_values('ClickThruRate')

fig = px.bar(df_bar_chart,
             x='Match type',
             y='ClickThruRate',
             title='Average Click-Through Rate by Match Type',
             labels={"ClickThruRate": "Average Click-Through Rate (%)"}
            )

# Display, save, and download plot
fig.show()
pio.write_image(fig, "Avg_CTR_rate_plot.png")
files.download("Avg_CTR_rate_plot.png")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>